# Predict

In [36]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")

import json
import pandas as pd

from rmgpy.data.thermo import ThermoDatabase, ThermoGroups

from thermo import ThermoGroups as SIDTThermoGroups
from utils import make_mol
from parameters import Ts

# Load test data

In [37]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")

In [38]:
with open("../data/splits/random.json", "r") as f:
    train_inds, test_inds = json.load(f)

In [39]:
test_df = hbi_unc_df.loc[test_inds, :]
test_df

,radical_smiles,radical_resonance_smiles,closed_shell_smiles,closed_shell_H298 (kcal/mol),closed_shell_S298 (cal/mol/K),closed_shell_Cp300 (cal/mol/K),closed_shell_Cp400 (cal/mol/K),closed_shell_Cp500 (cal/mol/K),closed_shell_Cp600 (cal/mol/K),closed_shell_Cp800 (cal/mol/K),...,unc_HBI_H298 (kcal/mol),unc_HBI_S298 (cal/mol/K),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
14,[O]C(=O)OC(=O)O,[O]C(=O)OC(=O)O,O=C(O)OC(=O)O,-223.695934,86.503270,24.694823,28.758100,32.040339,34.628195,38.156001,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
18,[O]OC(=O)OC(=O)O,[O]OC(=O)OC(=O)O,O=C(O)OC(=O)OO,-199.973245,94.255903,31.001150,34.739959,37.826260,40.355462,44.035249,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
25,C[C]1OC(=O)C(O)(C=O)O1,C[C]1OC(=O)C(O)(C=O)O1,CC1OC(=O)C(O)(C=O)O1,-206.467878,99.587328,33.632639,44.454340,53.571377,59.780918,67.788604,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
36,CC1(OO)COC(O[O])C(=O)O1,CC1(OO)COC(O[O])C(=O)O1,CC1(OO)COC(OO)C(=O)O1,-194.098719,108.407920,48.100822,61.544551,72.689006,80.724321,90.184054,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
49,[O]OC1(COO)COCC(=O)O1,[O]OC1(COO)COCC(=O)O1,O=C1COCC(COO)(OO)O1,-180.828929,108.787897,49.031759,62.443461,73.174034,81.237572,90.690096,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,[CH2]C#CC=C=C,C=C=[C]C=C=C,C=C=CC=C=C,96.710913,76.828171,26.525688,32.732134,37.443511,41.260269,47.258558,...,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2969,[CH2]C#CC=C=C,C=[C]C=C=C=C,C=C=C=CC=C,92.368996,78.947008,26.676979,32.843489,37.671520,41.507017,47.122008,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2979,C=C=C1[CH]C1=C,C=C=C1[CH]C1=C,C=C=C1CC1=C,112.767974,53.168531,24.008919,30.043266,34.942461,38.950145,44.884263,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2980,C=C=C1[CH]C1=C,C=[C]C1=CC1=C,C=CC1=CC1=C,118.596000,75.148273,23.377000,29.587000,34.875000,39.276000,46.044000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [41]:
mols = test_df["radical_resonance_smiles"].apply(make_mol)
mols = mols.to_list()

# Load thermo database

In [57]:
def make_prediction(thermo_database, tree, mol):
    atoms = {"*": atom for atom in mol.atoms if atom.radical_electrons==1}
    return thermo_database._add_group_thermo_data(None, tree, mol, atoms)[0]

In [42]:
thermo_database = ThermoDatabase()

# Predict with empirical tree

In [61]:
temp_columns = [column for column in test_df.columns if "HBI" in column]
test_result_df = test_df[["radical_resonance_smiles", *temp_columns]]

empirical_tree = ThermoGroups().load("/home/gridsan/hwpang/Software/RMG-database/input/thermo/groups/radical.py", thermo_database.local_context, thermo_database.global_context)

In [62]:
thermos = [make_prediction(thermo_database, empirical_tree, mol) for mol in mols]

In [63]:
test_result_df["empirical_H298 (kcal/mol)"] = [thermo.H298.value_si/4184 for thermo in thermos]
test_result_df["unc_empirical_H298 (kcal/mol)"] = [thermo.H298.uncertainty_si/4184 for thermo in thermos]
test_result_df["empirical_S298 (cal/mol/K)"] = [thermo.S298.value_si/4.184 for thermo in thermos]
test_result_df["unc_empirical_S298 (cal/mol/K)"] = [thermo.S298.uncertainty_si/4.184 for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"empirical_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.value_si[i]/4.184 for thermo in thermos]
    test_result_df[f"unc_empirical_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.uncertainty_si[i]/4.184 for thermo in thermos]
test_result_df

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A

,radical_resonance_smiles,HBI_H298 (kcal/mol),HBI_Sint298 (cal/mol/K),HBI_Cp300 (cal/mol/K),HBI_Cp400 (cal/mol/K),HBI_Cp500 (cal/mol/K),HBI_Cp600 (cal/mol/K),HBI_Cp800 (cal/mol/K),HBI_Cp1000 (cal/mol/K),HBI_Cp1500 (cal/mol/K),...,empirical_Cp500 (cal/mol/K),unc_empirical_Cp500 (cal/mol/K),empirical_Cp600 (cal/mol/K),unc_empirical_Cp600 (cal/mol/K),empirical_Cp800 (cal/mol/K),unc_empirical_Cp800 (cal/mol/K),empirical_Cp1000 (cal/mol/K),unc_empirical_Cp1000 (cal/mol/K),empirical_Cp1500 (cal/mol/K),unc_empirical_Cp1500 (cal/mol/K)
14,[O]C(=O)OC(=O)O,58.857481,-0.210403,-1.307188,-2.145019,-2.755436,-3.154323,-3.498782,-3.724675,-4.606060,...,-3.895793,0.0,-4.373805,0.0,-4.875717,0.0,-5.066922,0.0,-5.114723,0.0
18,[O]OC(=O)OC(=O)O,44.867361,-0.693252,-1.457701,-1.521980,-2.131394,-2.657387,-3.455461,-3.989172,-4.637430,...,-3.550000,0.0,-4.090000,0.0,-4.720000,0.0,-4.970000,0.0,-5.080000,0.0
25,C[C]1OC(=O)C(O)(C=O)O1,55.261992,-2.774457,5.119112,2.196657,-1.201741,-2.678547,-3.548237,-4.483736,-5.830052,...,-1.910000,0.0,-2.400000,0.0,-3.160000,0.0,-3.740000,0.0,-4.660000,0.0
36,CC1(OO)COC(O[O])C(=O)O1,56.585520,2.506289,-1.771977,-4.713521,-7.438894,-9.004805,-9.291943,-9.901793,-7.024417,...,-3.550000,0.0,-4.090000,0.0,-4.720000,0.0,-4.970000,0.0,-5.080000,0.0
49,[O]OC1(COO)COCC(=O)O1,48.234489,1.570257,-0.796301,-3.747607,-6.401129,-8.372562,-8.842852,-9.187769,-7.078146,...,-3.550000,0.0,-4.090000,0.0,-4.720000,0.0,-4.970000,0.0,-5.080000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,C=C=[C]C=C=C,33.569535,4.577665,-0.311584,-1.852373,-2.599189,-3.001817,-3.575296,-3.919648,-4.265209,...,-1.940000,0.0,-2.520000,0.0,-3.340000,0.0,-3.910000,0.0,-4.760000,0.0
2969,C=[C]C=C=C=C,37.911452,3.836255,-0.462875,-1.963729,-2.827198,-3.248566,-3.438746,-3.603371,-4.258011,...,-1.510000,0.0,-2.010000,0.0,-2.700000,0.0,-3.170000,0.0,-3.170000,0.0
2979,C=C=C1[CH]C1=C,36.202446,25.437539,1.176527,0.728440,0.047260,-0.493509,-0.957762,-1.128172,-1.774846,...,-1.880000,0.0,-1.890000,0.0,-2.200000,0.0,-2.600000,0.0,-2.600000,0.0
2980,C=[C]C1=CC1=C,30.374420,4.835224,1.808446,1.184706,0.114720,-0.819363,-2.117499,-2.850128,-4.706895,...,-1.510000,0.0,-2.010000,0.0,-2.700000,0.0,-3.170000,0.0,-3.170000,0.0


In [56]:
test_result_df.to_csv("../data/splits/test_empirical.csv", index=False)

# Predict with SIDT tree

In [64]:
temp_columns = [column for column in test_df.columns if "HBI" in column]
test_result_df = test_df[["radical_resonance_smiles", *temp_columns]]
test_result_df

sidt_tree = SIDTThermoGroups().load("../models/test/tree.py", thermo_database.local_context, thermo_database.global_context)

In [65]:
thermos = [make_prediction(thermo_database, sidt_tree, mol) for mol in mols]

In [66]:
test_result_df["sidt_H298 (kcal/mol)"] = [thermo.H298.value_si/4184 for thermo in thermos]
test_result_df["unc_sidt_H298 (kcal/mol)"] = [thermo.H298.uncertainty_si/4184 for thermo in thermos]
test_result_df["sidt_S298 (cal/mol/K)"] = [thermo.S298.value_si/4.184 for thermo in thermos]
test_result_df["unc_sidt_S298 (cal/mol/K)"] = [thermo.S298.uncertainty_si/4.184 for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"sidt_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.value_si[i]/4.184 for thermo in thermos]
    test_result_df[f"unc_sidt_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.uncertainty_si[i]/4.184 for thermo in thermos]
test_result_df

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A

,radical_resonance_smiles,HBI_H298 (kcal/mol),HBI_Sint298 (cal/mol/K),HBI_Cp300 (cal/mol/K),HBI_Cp400 (cal/mol/K),HBI_Cp500 (cal/mol/K),HBI_Cp600 (cal/mol/K),HBI_Cp800 (cal/mol/K),HBI_Cp1000 (cal/mol/K),HBI_Cp1500 (cal/mol/K),...,sidt_Cp500 (cal/mol/K),unc_sidt_Cp500 (cal/mol/K),sidt_Cp600 (cal/mol/K),unc_sidt_Cp600 (cal/mol/K),sidt_Cp800 (cal/mol/K),unc_sidt_Cp800 (cal/mol/K),sidt_Cp1000 (cal/mol/K),unc_sidt_Cp1000 (cal/mol/K),sidt_Cp1500 (cal/mol/K),unc_sidt_Cp1500 (cal/mol/K)
14,[O]C(=O)OC(=O)O,58.857481,-0.210403,-1.307188,-2.145019,-2.755436,-3.154323,-3.498782,-3.724675,-4.606060,...,0.412894,3.00000,-0.172027,3.00000,-1.04168,3.00000,-1.683180,3.00000,-3.705450,3.00000
18,[O]OC(=O)OC(=O)O,44.867361,-0.693252,-1.457701,-1.521980,-2.131394,-2.657387,-3.455461,-3.989172,-4.637430,...,2.161240,3.00000,1.702480,3.00000,1.26215,3.00000,-0.928170,3.00000,-3.176200,3.00000
25,C[C]1OC(=O)C(O)(C=O)O1,55.261992,-2.774457,5.119112,2.196657,-1.201741,-2.678547,-3.548237,-4.483736,-5.830052,...,2.714390,3.00000,2.346110,3.00000,1.17847,3.00000,0.282207,3.00000,-0.997631,3.00000
36,CC1(OO)COC(O[O])C(=O)O1,56.585520,2.506289,-1.771977,-4.713521,-7.438894,-9.004805,-9.291943,-9.901793,-7.024417,...,2.161240,3.00000,1.702480,3.00000,1.26215,3.00000,-0.928170,3.00000,-3.176200,3.00000
49,[O]OC1(COO)COCC(=O)O1,48.234489,1.570257,-0.796301,-3.747607,-6.401129,-8.372562,-8.842852,-9.187769,-7.078146,...,2.161240,3.00000,1.702480,3.00000,1.26215,3.00000,-0.928170,3.00000,-3.176200,3.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,C=C=[C]C=C=C,33.569535,4.577665,-0.311584,-1.852373,-2.599189,-3.001817,-3.575296,-3.919648,-4.265209,...,-2.090000,6.67512,-1.960080,6.26049,-1.54962,6.78152,-1.614510,7.29506,-1.695440,7.14694
2969,C=[C]C=C=C=C,37.911452,3.836255,-0.462875,-1.963729,-2.827198,-3.248566,-3.438746,-3.603371,-4.258011,...,-2.090000,6.67512,-1.960080,6.26049,-1.54962,6.78152,-1.614510,7.29506,-1.695440,7.14694
2979,C=C=C1[CH]C1=C,36.202446,25.437539,1.176527,0.728440,0.047260,-0.493509,-0.957762,-1.128172,-1.774846,...,-2.090000,6.67512,-1.960080,6.26049,-1.54962,6.78152,-1.614510,7.29506,-1.695440,7.14694
2980,C=[C]C1=CC1=C,30.374420,4.835224,1.808446,1.184706,0.114720,-0.819363,-2.117499,-2.850128,-4.706895,...,-2.090000,6.67512,-1.960080,6.26049,-1.54962,6.78152,-1.614510,7.29506,-1.695440,7.14694


In [67]:
split = "random"
run_name = "test"
test_result_df.to_csv(f"../models/split-{split}_run-{run_name}/test.csv", index=False)